In [5]:
%matplotlib inline

import datacube
import datetime
import fiona
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio.mask
import rasterio.features
from shapely import geometry
import seaborn as sns
import sys
import xarray as xr

import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from datacube.storage import masking
from datacube.utils import geometry
from datacube.helpers import write_geotiff, ga_pq_fuser

sys.path.append('LCCS_wet_veg/Scripts')
import dea_bandindices
from dea_datahandling import load_ard

dc = datacube.Datacube(app='tcw')

# %load_ext autoreload

# %autoreload 2

In [4]:
pwd

'/home/jovyan/dev/dea-notebooks/Gabrielle_dev'

In [20]:
# Define area of interest
query = {
    
        #Mathoura Large
# 'y': (-35.6545, -36.0320),
# 'x': (144.8179, 145.4289), 
    
    
            #Moria Lakes
# 'y': (-35.9634, -35.8515),
# 'x': (144.8887, 145.0738),
    
                #lake Frome
# 'y': (-30.354, -30.889),
# 'x': (139.482, 140.032),

        #lake carnegie
# 'y': (-36.900, -36.988),
# 'x': (149.800, 149.925),

            #lake carnegie Smol
'y': (-26.076, -26.352),
'x': (122.216, 122.552),
    
    
'time': ('2016-01-01', '2016-12-31'),
'crs': 'EPSG:4326',
'output_crs': 'EPSG: 4326',
'resolution': (-25, 25)
}

In [21]:
# list(dc.list_products()['name'])

In [22]:
#load data
data_lst8 = dc.load(product='ga_ls8c_ard_3', group_by='solar_day', **query)

CRSError: Invalid projection: EPSG: 4326: (Internal Proj Error: proj_create: crs not found)

In [ ]:
data_lst8.isel(time=0)

In [ ]:
# Total obs
total_obs = int(len(data_lst8.time))
total_obs

In [ ]:
# Turn terrain masked values from -999 to Nan
# Is this done already from the load_ard? original code was dc.load
want_to_mask = (data_lst8 == -999).to_array(dim='variables').all(dim='variables')
terrain_out = data_lst8.where(~want_to_mask,np.NaN)

In [9]:
# Load PQ
sensor_pq = dc.load(product='ga_ls8c_ard_3',fuse_func=ga_pq_fuser,group_by='solar_day',**query)
sensor_pq


<xarray.Dataset>
Dimensions:                     (time: 23, x: 1418, y: 1335)
Coordinates:
  * time                        (time) datetime64[ns] 2016-01-11T01:45:23.962938 ... 2016-12-28T01:45:33.675150
  * y                           (y) float64 -2.858e+06 -2.858e+06 ... -2.891e+06
  * x                           (x) float64 -9.662e+05 -9.662e+05 ... -9.308e+05
Data variables:
    nbar_coastal_aerosol        (time, y, x) int16 437 437 433 ... 435 430 424
    nbar_blue                   (time, y, x) int16 499 499 497 ... 515 511 509
    nbar_green                  (time, y, x) int16 842 842 845 ... 914 922 909
    nbar_red                    (time, y, x) int16 1608 1608 1666 ... 1777 1749
    nbar_nir                    (time, y, x) int16 2425 2425 2517 ... 2279 2258
    nbar_swir_1                 (time, y, x) int16 3525 3525 3666 ... 3737 3667
    nbar_swir_2                 (time, y, x) int16 3151 3151 3246 ... 2934 2897
    nbar_panchromatic           (time, y, x) int16 1087 1114 1210 ... 1230 1193
    nbart_coastal_aerosol       (time, y, x) int16 434 434 430 ... 429 426 422
    nbart_blue                  (time, y, x) int16 495 495 494 ... 508 507 507
    nbart_green                 (time, y, x) int16 835 835 840 ... 902 915 905
    nbart_red                   (time, y, x) int16 1595 1595 1653 ... 1763 1741
    nbart_nir                   (time, y, x) int16 2405 2405 2499 ... 2262 2248
    nbart_swir_1                (time, y, x) int16 3494 3494 3638 ... 3708 3649
    nbart_swir_2                (time, y, x) int16 3123 3123 3221 ... 2910 2882
    nbart_panchromatic          (time, y, x) int16 1077 1107 1202 ... 1223 1189
    oa_fmask                    (time, y, x) uint8 1 1 1 1 1 1 1 ... 1 1 1 1 1 1
    oa_nbar_contiguity          (time, y, x) uint8 1 1 1 1 1 1 1 ... 1 1 1 1 1 1
    oa_nbart_contiguity         (time, y, x) uint8 1 1 1 1 1 1 1 ... 1 1 1 1 1 1
    oa_azimuthal_exiting        (time, y, x) float32 123.91837 ... -142.61702
    oa_azimuthal_incident       (time, y, x) float32 155.1052 ... -108.25658
    oa_combined_terrain_shadow  (time, y, x) uint8 1 1 1 1 1 1 1 ... 1 1 1 1 1 1
    oa_exiting_angle            (time, y, x) float32 3.8653843 ... 0.39266902
    oa_incident_angle           (time, y, x) float32 29.69296 ... 28.07935
    oa_relative_azimuth         (time, y, x) float32 9.729019 ... 8.473663
    oa_relative_slope           (time, y, x) float32 31.186821 ... 34.360443
    oa_satellite_azimuth        (time, y, x) float32 99.130714 ... 99.15083
    oa_satellite_view           (time, y, x) float32 4.244291 ... 0.8872468
    oa_solar_azimuth            (time, y, x) float32 89.401695 ... 90.67717
    oa_solar_zenith             (time, y, x) float32 30.629576 ... 28.633675
    oa_time_delta               (time, y, x) float32 2.325597 ... 6.5816975
Attributes:
    crs:      EPSG: 3577

In [10]:
# Load wofs filtered summary
wofs_filtered_summary = dc.load(product='wofs_annual_summary',**query)
wofs_filtered_summary

<xarray.Dataset>
Dimensions:      (time: 1, x: 1418, y: 1335)
Coordinates:
  * time         (time) datetime64[ns] 2016-01-01
  * y            (y) float64 -2.858e+06 -2.858e+06 ... -2.891e+06 -2.891e+06
  * x            (x) float64 -9.662e+05 -9.662e+05 ... -9.308e+05 -9.308e+05
Data variables:
    count_wet    (time, y, x) int16 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    count_clear  (time, y, x) int16 27 27 26 26 26 26 26 ... 34 34 34 34 34 35
    frequency    (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    crs:      EPSG: 3577

In [11]:
# Mask cloud

# Make mask
good_quality = masking.make_mask(sensor_pq.pixelquality,
                                 cloud_acca='no_cloud',
                                 cloud_fmask='no_cloud',
                                 cloud_shadow_acca='no_cloud_shadow',
                                 cloud_shadow_fmask='no_cloud_shadow',
                                 blue_saturated=False,
                                 green_saturated=False,
                                 red_saturated=False,
                                 nir_saturated=False,
                                 swir1_saturated=False,
                                 swir2_saturated=False,
                                 contiguous=True)

# Apply mask 
Masked_lst8_data = terrain_out.where(good_quality)

AttributeError: 'Dataset' object has no attribute 'pixelquality'

In [12]:
data_lst8_once = data_lst8.isel(time=3)
data_lst8_once
#data_lst8_once = Masked_lst8_data.isel(time=3)

<xarray.Dataset>
Dimensions:                     (x: 1418, y: 1335)
Coordinates:
    time                        datetime64[ns] 2016-02-28T01:45:13.360583
  * y                           (y) float64 -2.858e+06 -2.858e+06 ... -2.891e+06
  * x                           (x) float64 -9.662e+05 -9.662e+05 ... -9.308e+05
Data variables:
    nbar_coastal_aerosol        (y, x) int16 461 461 455 455 ... 416 417 410 404
    nbar_blue                   (y, x) int16 522 522 520 529 ... 494 500 490 486
    nbar_green                  (y, x) int16 917 917 933 947 ... 872 881 879 869
    nbar_red                    (y, x) int16 1735 1735 1778 ... 1686 1676 1642
    nbar_nir                    (y, x) int16 2799 2799 2802 ... 2269 2283 2251
    nbar_swir_1                 (y, x) int16 3698 3698 3786 ... 3674 3593 3505
    nbar_swir_2                 (y, x) int16 3249 3249 3447 ... 2811 2817 2745
    nbar_panchromatic           (y, x) int16 1147 1163 1328 ... 1155 1133 1100
    nbart_coastal_aerosol       (y, x) int16 454 454 450 450 ... 406 410 406 402
    nbart_blue                  (y, x) int16 514 514 513 523 ... 482 492 485 483
    nbart_green                 (y, x) int16 903 903 921 938 ... 852 866 870 863
    nbart_red                   (y, x) int16 1705 1705 1752 ... 1654 1658 1631
    nbart_nir                   (y, x) int16 2756 2756 2766 ... 2230 2260 2237
    nbart_swir_1                (y, x) int16 3635 3635 3731 ... 3604 3553 3482
    nbart_swir_2                (y, x) int16 3190 3190 3395 ... 2754 2784 2725
    nbart_panchromatic          (y, x) int16 1125 1148 1310 ... 1140 1123 1095
    oa_fmask                    (y, x) uint8 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    oa_nbar_contiguity          (y, x) uint8 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    oa_nbart_contiguity         (y, x) uint8 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    oa_azimuthal_exiting        (y, x) float32 124.08696 ... -138.15915
    oa_azimuthal_incident       (y, x) float32 177.83429 177.83429 ... -84.80056
    oa_combined_terrain_shadow  (y, x) uint8 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    oa_exiting_angle            (y, x) float32 3.8869057 ... 0.48132652
    oa_incident_angle           (y, x) float32 35.984886 35.984886 ... 36.700302
    oa_relative_azimuth         (y, x) float32 31.271477 31.271477 ... 31.975868
    oa_relative_slope           (y, x) float32 53.74733 53.74733 ... 53.35859
    oa_satellite_azimuth        (y, x) float32 99.13075 99.13075 ... 99.15098
    oa_satellite_view           (y, x) float32 4.268114 4.268114 ... 0.9829216
    oa_solar_azimuth            (y, x) float32 67.859276 67.859276 ... 67.17511
    oa_solar_zenith             (y, x) float32 37.398277 37.398277 ... 37.24538
    oa_time_delta               (y, x) float32 2.3331482 2.3331482 ... 6.611895
Attributes:
    crs:      EPSG: 3577

In [13]:

#Prepare geomedian imagery for plotting

rgb = data_lst8_once.to_array(dim='color').sel(color=[
    'red','green', 'blue']).transpose('y', 'x', 'color')
fake_saturation = 2000.0  #you may need to adjust this to get a clear image. higher numbers will make it darker, 
                                                                            #lower will make it ligher
rgb = rgb.astype('double')
clipped_visible = rgb.where(rgb<fake_saturation).fillna(fake_saturation)
max_val = clipped_visible.max(['y', 'x'])
scaled = (clipped_visible / max_val)

fig = plt.figure(figsize =(15,15)) #edit size of plot 
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05) #set border dimensions
fig.patch.set_facecolor('white') #make border white 
fig.patch.set_alpha(0.99)#ensure border white
plt.axis('off')#remove axis 
plt.title('True Colour') #add title
plt.imshow(scaled, interpolation = 'nearest') #create image

KeyError: "not all values found in index 'color'"

In [ ]:
#Do tassled Cap transformation
TCI = BandIndices.tasseled_cap(Masked_lst8_data, tc_bands=['wetness'], drop=True)

tcw = TCI['wetness']

In [ ]:
# Plot an example of TCW
tcw.isel(time=3).plot(cmap='viridis', figsize =(15,12),vmin=-300)

In [ ]:
#make Fisher index

lst_fisher = 1.7204 +((171*Masked_lst8_data['green'])+(3*Masked_lst8_data['red'])-(70*Masked_lst8_data['nir'])-(45*Masked_lst8_data['swir1'])-(71*Masked_lst8_data['swir2']))/10000
lst_fisher

In [ ]:
# Plot an example of Fisher Index
lst_fisher.isel(time=3).plot(cmap='viridis', figsize =(15,12), vmin=-10)

In [ ]:
inundated_veg_msk = (np.logical_and(lst_fisher>=-10,tcw>=-400))
inundated_veg = lst_fisher.where(inundated_veg_msk==True,0)
inundated_veg = inundated_veg.where(inundated_veg_msk==False,1)

In [ ]:
# count_wet = inundated_veg.where(inundated_veg==1)
count_wet = inundated_veg.where(inundated_veg>0).count(dim=['x','y'])

In [ ]:
count_wet.plot(figsize =(15,12))

In [ ]:
wet_frequency = inundated_veg.sum(dim='time')

In [ ]:
# Plot inundated veg
wet_frequency.plot(cmap='viridis', figsize =(15,12))

In [8]:
# Plot wofs comparison
wofs_filtered_summary.frequency.plot(cmap='viridis',figsize =(15,12),vmin=0.2,)

In [ ]:
precent_threhold = total_obs*0.2
precent_threhold

In [ ]:
persistant_wet = wet_frequency.where(wet_frequency>precent_threhold,0)
persistant_wet = persistant_wet.where(wofs_filtered_summary.frequency<0.2,0)
persistant_wet.plot(figsize =(15,12))

In [ ]:
persistant_wet